In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

df = catalog.load('raw/openaire/researchproduct_dev#parquet')

                    INFO     Loading data from raw/openaire/researchproduct_dev#parquet         ]8;id=713340;file:///root/kedro-unlp/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=455636;file:///root/kedro-unlp/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [2]:
def _pick_load_dt(df: pd.DataFrame):
    # Si hay una sola fecha en el batch, usala; si hay varias, quedate con la más reciente;
    # si no hay, hoy.
    if 'load_datetime' not in df.columns or df['load_datetime'].isna().all():
        return date.today()
    vals = df['load_datetime'].dropna()
    if vals.nunique() == 1:
        return vals.iloc[0]
    return pd.to_datetime(vals).max().date()

In [3]:
df

,authors,openAccessColor,publiclyFunded,type,language,countries,subjects,mainTitle,subTitle,descriptions,publicationDate,publisher,embargoEndDate,sources,formats,contributors,coverages,bestAccessRight,container,documentationUrls,codeRepositoryUrl,programmingLanguage,contactPeople,contactGroups,tools,size,version,geoLocations,id,originalIds,pids,dateOfCollection,lastUpdateTimeStamp,indicators,projects,organizations,communities,collectedFrom,instances,isGreen,isInDiamondJournal,relOrganizationId,load_datetime
0,"[{'fullName': 'Susevich, María Laura', 'name':...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Dicistrovirus from the pollinator community fo...,None,[Los Dicistrovirus son una familia de virus qu...,2022-08-31,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::319dc88111c9b2d6021228590e79130a,[50|4dc99724cf04::319dc88111c9b2d6021228590e79...,None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,"[{'acronym': 'UNLPAM', 'id': 'openorgs____::db...","[{'code': 'knowmad', 'label': 'Knowmad Institu...",[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-05
1,"[{'fullName': 'Steffen, Kevin Denis', 'name': ...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Variability in the growth rates of Saanen kids...,None,[This study was carried out to determine the v...,2023-12-18,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"[oai:ojs.csi.unmsm:article/25240, 50|4dc99724c...",None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,[{'acronym': 'Universidad Nacional de La Pampa...,None,[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,"[{'fullName': 'Giorgi, Edgard Ervar Salvador',...",None,False,publication,"{'code': 'eng', 'label': 'English'}","[{'code': 'AR', 'label': 'Argentina', 'provena...","[{'provenance': None, 'subject': {'scheme': 'k...",A case study of oversight and neglect in the h...,None,[Presentamos y discutimos datos obtenidos de l...,2024-01-01,None,None,None,"[application/pdf, 114-117]",None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",None,None,None,None,None,None,None,None,None,None,dedup_wf_002::037c3a3ae3212005fc63e98d8378595c,"[oai:sedici.unlp.edu.ar:10915/170737, 50|od___...",None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,"[{'acronym': 'UNLP', 'id': 'openorgs____::40b9...",None,[{'key': 'openaire____::df45502607927471ecf8a6...,"[{'accessRight': None, 'alternateIdentifiers':...",True,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-05
999,"[{'fullName': 'Jong-youn, Choo', 'name': 'Choo...",gold,False,publication,"{'code': 'esl/spa', 'label': 'Spanish'}","[{'code': 'AR', 'label': 'Argentina', 'provena...","[{'provenance': None, 'subject': {'scheme': 'k...",Un nuevo enfoque de la relación Corea-Argentina,None,"[El autor señala que Argentina y Corea, despué...",2007-01-01,Universidad Nacional de La Plata,None,"[Relaciones Internacionales, V

In [4]:
load_dt = _pick_load_dt(df)

In [5]:
df_research_pid = df.loc[:,['id','pids']]
df_research_pid.dropna(inplace=True)

In [6]:
df_research_pid

,id,pids
3,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,"[{'scheme': 'handle', 'value': '10261/97737'}]"
116,dedup_wf_002::0005b301e367f0fc799b29dc387d3bd1,"[{'scheme': 'handle', 'value': '11336/112574'}]"
...,...,...
987,dedup_wf_002::0371facbe40925273620155e4639ff06,"[{'scheme': 'handle', 'value': '20.500.12110/a..."
989,dedup_wf_002::03758bd23c38a7125705d74fc4f38575,"[{'scheme': 'handle', 'value': '11336/33252'}]"


In [7]:
df_research_pid = df_research_pid.explode('pids').reset_index(drop=True)

In [8]:
df_research_pid

,id,pids
0,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,"{'scheme': 'handle', 'value': '10261/97737'}"
1,dedup_wf_002::0005b301e367f0fc799b29dc387d3bd1,"{'scheme': 'handle', 'value': '11336/112574'}"
...,...,...
146,dedup_wf_002::0371facbe40925273620155e4639ff06,"{'scheme': 'handle', 'value': '20.500.12110/af..."
147,dedup_wf_002::03758bd23c38a7125705d74fc4f38575,"{'scheme': 'handle', 'value': '11336/33252'}"


In [9]:
df_pid = pd.json_normalize(df_research_pid['pids'])

In [10]:
df_pid

,scheme,value
0,handle,10261/97737
1,handle,11336/112574
...,...,...
146,handle,20.500.12110/afa_v12_n01_p159
147,handle,11336/33252


In [11]:
df_research_pid = pd.concat([df_research_pid['id'], df_pid], axis=1)
df_research_pid['load_datetime'] = load_dt

In [12]:
df_research_pid

,id,scheme,value,load_datetime
0,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,handle,10261/97737,2025-09-05
1,dedup_wf_002::0005b301e367f0fc799b29dc387d3bd1,handle,11336/112574,2025-09-05
...,...,...,...,...
146,dedup_wf_002::0371facbe40925273620155e4639ff06,handle,20.500.12110/afa_v12_n01_p159,2025-09-05
147,dedup_wf_002::03758bd23c38a7125705d74fc4f38575,handle,11336/33252,2025-09-05


## Node
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [13]:
def openaire_land_researchproduct_pids(df: pd.DataFrame)-> pd.DataFrame:

    df_research_pid = df.loc[:,['id','pids']]
    df_research_pid.dropna(inplace=True)
    
    df_research_pid = df_research_pid.explode('pids').reset_index(drop=True)

    df_pid = pd.json_normalize(df_research_pid['pids'])
    
    df_research_pid = pd.concat([df_research_pid['id'], df_pid], axis=1)
    df_research_pid['load_datetime'] = load_dt

    return df_research_pid

In [14]:
df_research_pid = openaire_land_researchproduct_pids(df)

In [15]:
df_research_pid

,id,scheme,value,load_datetime
0,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,handle,10261/97737,2025-09-05
1,dedup_wf_002::0005b301e367f0fc799b29dc387d3bd1,handle,11336/112574,2025-09-05
...,...,...,...,...
146,dedup_wf_002::0371facbe40925273620155e4639ff06,handle,20.500.12110/afa_v12_n01_p159,2025-09-05
147,dedup_wf_002::03758bd23c38a7125705d74fc4f38575,handle,11336/33252,2025-09-05
